In [1]:
import pandas as pd
import numpy as np

# Tweets

In [2]:
import twint
import nest_asyncio
nest_asyncio.apply()
def downloadTweets(target, limit=100000):
    # Configuration
    config = twint.Config()

    # Function Parameters
    config.Search = target
    config.Limit = limit

    # High-quality Tweets selection
    config.Lang = 'en'
    config.Filter_retweets = True
    config.Popular_tweets = True
    config.Min_likes = 50
    
    # Output 
    config.Pandas = True
    config.Hide_output = True
    # config.Store_csv = True
    # config.Output = 'downloadTweets.csv'

    # Search
    twint.run.Search(config)
    df = twint.storage.panda.Tweets_df
    return df

In [3]:
rawTweets = downloadTweets("Bitcoin")

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.


In [4]:
rawTweets

,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,cashtags,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1572051372597563394,1572051372597563394,1.663641e+12,2022-09-19 22:33:49,-0400,,"I WILL GIVE $1,000 #BITCOIN TO MY FOLLOWERS ...",en,[bitcoin],[],...,,,,,,[],,,,
1,1572049472695472130,1572049472695472130,1.663641e+12,2022-09-19 22:26:16,-0400,,"5 people now ! post your cashapp, PayPal, Bitcoin",en,[],[],...,,,,,,[],,,,
2,1572044178502074368,1572044178502074368,1.663640e+12,2022-09-19 22:05:13,-0400,,The #Bitcoin rainbow price chart which maxis u...,en,[bitcoin],[btc],...,,,,,,[],,,,
3,1572043586559934472,1572043586559934472,1.663639e+12,2022-09-19 22:02:52,-0400,,“Why #Bitcoin Has Value”—clip from my recent a...,en,[bitcoin],[],...,,,,,,[],,,,
4,1572041666294431746,1572041666294431746,1.663639e+12,2022-09-19 21:55:15,-0400,,Remember: Kings and Queens don't pay inheritan...,en,[],[],...,,,,,,[],,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10938,1568707306610720771,1568707306610720771,1.662844e+12,2022-09-10 17:05:41,-0400,,|￣￣￣￣￣￣￣￣￣ | | CONSTRUYE UN | | ...,es,[bitcoin],[],...,,,,,,[],,,,
10939,1568706733442301953,1568706733442301953,1.662844e+12,2022-09-10 17:03:25,-0400,,At the Hal Finney Cafeteria today with Gabriel...,en,[bitcoin],[],...,,,,,,[],,,,
10940,1568705655904653315,1568705655904653315,1.662844e+12,2022-09-10 16:59:08,-0400,,İyi Geceler Dostlar!🌠 Yemyeşil Bir Sabaha Uyan...,tr,"[kripto, bitcoin, ethereum]",[],...,,,,,,[],,,,
10941,1568704213231755264,1568704213231755264,1.662843e+12,2022-09-10 16:53:24,-0400,,İyi geceler #Crypto 🧡 Sabah ola hayrola🙌 Mutlu...,tr,"[crypto, btc, bitcoin, ethereum, eth, kripto, ...",[btc],...,,,,,,[],,,,


In [5]:
rawTweets.shape

(10943, 38)

Limited Data retrived.

# Reddit

In [6]:
import praw
import json

In [7]:
CLIENT_ID = 'wD4tJp9o4AER28jDR6WltQ'
SECRET_TOKEN = '7bVuyifIw4dAsi7WE58xrALGIp2pAA'

reddit = praw.Reddit(
    client_id = CLIENT_ID,
    client_secret = SECRET_TOKEN,
    user_agent = '',
    passcode = '',
    username = '',
)


In [8]:
print(reddit.user.me())

None


In [9]:
import requests

auth = requests.auth.HTTPBasicAuth(CLIENT_ID, SECRET_TOKEN)

In [10]:
data = {'grant_type': 'password',
        'username': '',
        'password': ''}

headers = {'User-Agent':'testscript by u/fakebot3' }

In [11]:
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

In [12]:
TOKEN = res.json()

In [13]:
TOKEN

{'error': 'invalid_grant'}